In [ ]:
import numpy as np
from tqdm import tqdm
from scipy import stats
from scipy.special import comb
from matplotlib import pyplot as plt

# Frequentist

In [ ]:
sum_list = []
for my_lambda in np.arange(.01, .5, .000001):
    cur_sum = 0
    true_count = 0
    for r_tilde in range(0, 13):
        val = .5**12/((r_tilde/12)**r_tilde * (1-r_tilde/12)**(12-r_tilde))
        if .5**12/((r_tilde/12)**r_tilde * (1-r_tilde/12)**(12-r_tilde)) <= my_lambda:
            true_count+=1
            cur_sum = cur_sum + comb(12, r_tilde) *.5**12
    sum_list.append(cur_sum)

In [ ]:
plt.figure()
plt.title('SE vs lambda')
plt.xlabel('lambda')
plt.ylabel('SE')
plt.plot([i for i  in np.arange(.01, .5, .000001)], [(elem-.05)**2  for elem in sum_list])
plt.show()

In [ ]:
xs = [i for i  in np.arange(.01, .5, .000001)]
ys = [(elem - .05)**2 for elem in sum_list]
relevant = [xs[i] for i in range(len(xs)) if ys[i] < .0019]

In [ ]:
print(np.min(relevant))
print(np.max(relevant))

# Bayesian

In [ ]:
def integrate(lower, upper, dx, exp_fn):
    my_sum = 0
    for x in np.arange(lower, upper, dx):
        my_sum += exp_fn(x) * dx
    return my_sum

In [ ]:
def integrand_factory(r_tilde):
    def integrand(k):
        return k**r_tilde * (1-k)**(12-r_tilde)
    return integrand

In [ ]:
integrand_lookups = {}
for r_tilde in tqdm(range(0, 13)):
    integrand = integrand_factory(r_tilde)
    result = integrate(0, .5, .0001,  integrand)
    integrand_lookups[r_tilde] = result

In [ ]:
def eval_indicator(r_tilde, my_lambda):
    numerator = np.min([r_tilde/12, .5])**r_tilde * (1 - np.min([r_tilde/12, .5]))**(12-r_tilde)
    denominator = np.max([r_tilde/12, 7/12])**r_tilde * (1 - np.max([r_tilde/12, 7/12]))**(12-r_tilde)
    return numerator/denominator < my_lambda

def eval_at_lambda(integrand_lookups, my_lambda):
    my_sum = 0
    for r_tilde in range(0, 13):
        include = eval_indicator(r_tilde, my_lambda)
        if include:
            my_sum = my_sum + comb(12, r_tilde) * integrand_lookups[r_tilde]
    return my_sum

In [ ]:
out = [eval_at_lambda(integrand_lookups, i) for i in np.arange(.1, 1.9, .01)]

In [ ]:
plt.figure()
plt.title('SE vs lambda')
plt.xlabel('lambda')
plt.ylabel('SE')
plt.plot([i for i  in np.arange(.1, 1.9, .01)], [(elem - .05)**2 for elem in out])
plt.show()

In [ ]:
xs = [i for i  in np.arange(.1, 1.9, .01)]
ys = [(elem - .05)**2 for elem in out]
relevant = [xs[i] for i in range(len(xs)) if ys[i] < .0005]

In [ ]:
print(np.min(relevant))
print(np.max(relevant))

# 2

In [ ]:
def monte_carlo_sim(n, iterations):
    x_0 = []
    x_1 = []
    for _ in range(n//2):
        x_0.append(np.random.normal(-.25, 1))
        
    for _ in range(n//2):
        x_1.append(np.random.normal(.25, 1))
        
    xbar_0 = np.mean(x_0)
    xbar_1 = np.mean(x_1)
    
    xs = []
    ys = []
    for _ in range(iterations):
        if np.random.uniform(0, 1) < .95:
            ys.append(0)
            xs.append(np.random.normal(-.25, 1))
        else:
            ys.append(1)
            xs.append(np.random.normal(.25, 1))
            
    #assumed equal priors
    correct = 0
    for idx, x in enumerate(xs):
        y = ys[idx]
        if np.abs(xbar_0 - x) < np.abs(xbar_1 - x):
            if y == 0:
                correct+=1
        else:
            if y == 1:
                correct+=1
                
    bad_prior_acc = correct/iterations
    
    #known correct priors
    correct=0
    for idx, x in enumerate(xs):
        y = ys[idx]
        if stats.norm(xbar_0, 1).pdf(x) * .95 > stats.norm(xbar_1, 1).pdf(x) * .05:
            if y == 0:
                correct+=1
        else:
            if y == 1:
                correct +=1
                
    good_prior_acc = correct/iterations
    return bad_prior_acc, good_prior_acc

In [ ]:
bad_prior_accs = []
good_prior_accs = []
for replication in tqdm(range(100)):
    bad_prior_acc, good_prior_acc = monte_carlo_sim(128, 250)
    bad_prior_accs.append(bad_prior_acc)
    good_prior_accs.append(good_prior_acc)

In [ ]:
plt.figure(figsize=(7, 5))
plt.title('Monte Carlo Accuracy')
plt.hist(bad_prior_accs, label='Estimated Prior')
plt.hist(good_prior_accs, label='Correct Prior')
plt.legend()
plt.show()

In [ ]:
stats.ranksums(bad_prior_accs, good_prior_accs)

# 3

In [ ]:
def monte_carlo_sim_2(n, iterations):
    x_0 = []
    x_1 = []
    for _ in range(int(n * .95)):
        x_0.append(np.random.normal(-1, 5))
    
    for _ in range(int(n * .05)):
        x_1.append(np.random.normal(1, 5))
        
    xbar_0 = np.mean(x_0)
    xbar_1 = np.mean(x_1)
    
    xs = []
    ys = []
    for _ in range(iterations):
        if np.random.uniform(0, 1) < .95:
            ys.append(0)
            xs.append(np.random.normal(-1, 5))
        else:
            ys.append(1)
            xs.append(np.random.normal(1, 5))
            
    #random chance
    correct=0
    for idx in range(len(xs)):
        if np.random.uniform(0, 1) < .5:
            if ys[idx] == 0:
                correct+=1 
        else:
            if ys[idx] == 1:
                correct+=1 
    rand_correct = correct/iterations
    
    
    #est correct priors
    correct=0
    for idx, x in enumerate(xs):
        y = ys[idx]
        if stats.norm(xbar_0, 5).pdf(x) * .95 > stats.norm(xbar_1, 5).pdf(x) * .05:
            if y == 0:
                correct+=1
        else:
            if y == 1:
                correct +=1
                
    est_prior_acc = correct/iterations
    return rand_correct, est_prior_acc

In [ ]:
rand_accs = []
est_prior_accs = []
for replication in tqdm(range(100)):
    rand_acc, est_prior_acc = monte_carlo_sim_2(15, 250)
    rand_accs.append(rand_acc)
    est_prior_accs.append(est_prior_acc)

In [ ]:
plt.figure(figsize=(7, 5))
plt.title('Monte Carlo Accuracy')
plt.hist(est_prior_accs, label='Bayes Plug In Correct Prior')
plt.hist(rand_accs, label='Random Selection')
plt.legend()
plt.show()

In [ ]:
stats.ranksums(est_prior_accs, rand_accs)